In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ares_params import ares_params, redshifts

In [2]:
low_bound = np.array([3, 37, 3, 0.05])
up_bound = np.array([6, 40, 6, 2])

#deviation of change for each param
#dev = [0.00008, 0.00008, 0.00008, 0.00008]
#dev = []
dev = 0.01 * (up_bound - low_bound)

In [ ]:
new_param = normal(chain[i-1, :], dev)

In [3]:
dev

array([0.03  , 0.03  , 0.03  , 0.0195])

# Functions

In [4]:
def normalize(arr): # to normalize the parameter range
    #input: an array
    #output: normalized array
    
	op = np.empty(arr.shape)
	for i in range(arr.shape[1]):
		if ares_params[i][3] == "lin":
			op[:,i] = (arr[:,i] - ares_params[i][1])/(ares_params[i][2] - ares_params[i][1])
		elif ares_params[i][3] == "log":
			op[:,i] = (np.log10(arr[:,i]/ares_params[i][1]))/(np.log10(ares_params[i][2]/ares_params[i][1])) 
		else:
			raise ValueError("Invalid normalization type in ares_params")
	return op

def denormalize (arr): # to denormalize the parameter range
    #input: an array
    #output: denormalized array
    
	op = np.empty(arr.shape)
	for i in range(arr.shape[1]):
		if ares_params[i][3] == "lin":
			op[:,i] = arr[:,i] * (ares_params[i][2] - ares_params[i][1]) + ares_params[i][1]
		elif ares_params[i][3] == "log":
			op[:,i] = 10.0**(arr[:,i] * (np.log10(ares_params[i][2]) - np.log10(ares_params[i][1])) + np.log10(ares_params[i][1]))
		else:
			raise ValueError("Invalid normalization type in ares_params")
	return op

In [5]:
def check_normalize(new_param): #to check if the new params exceed the reasonable limits or not (e.g f_Star>1)
    x = new_param.shape[0]
    y = new_param.shape[1]   
    #print( x, y)
    check_array=np.zeros((x, y), dtype=bool)
    result = True
        
    for i in range(x):
        for j in range(y):
            #print(i)
            #print(j)
            if new_param[i, j] <= 1:
            
                if new_param[i, j] >= 0:
                    check_array[i, j] = True
                    
                else: 
                    check_array[i, j] = False
            else: 
                check_array[i, j] = False
        
            result = result & check_array[i, j] 
        
        
    return result

In [6]:
def check_limits(new_param, low, up): #to check if the new params exceed the reasonable limits or not (e.g f_Star>1)
    x = new_param.shape[0]
    y = new_param.shape[1]   
    #print( x, y)
    check_array=np.zeros((x, y), dtype=bool)
    result = True
    counter = 0   
    for i in range(x):
        for j in range(y):
            #print(i)
            #print(j)
            if new_param[i, j] <= up[j]:
            
                if new_param[i, j] >= low[j]:
                    check_array[i, j] = True
                    
                else: 
                    check_array[i, j] = False
                    counter = counter +1 
            else: 
                check_array[i, j] = False
                counter = counter +1 
        
            result = result & check_array[i, j] 
        
        
    return result, counter

# Load Data 

In [7]:
df1 = pd.read_csv("mcmc_params.txt", sep=" ", header=None)

In [8]:
df1

0         1         2         3
0        0.333333  0.270346  0.270346  0.487179
1        0.331996  0.272468  0.275850  0.486696
2        0.331996  0.272468  0.275850  0.486696
3        0.334644  0.280457  0.276013  0.486887
4        0.334644  0.280457  0.276013  0.486887
...           ...       ...       ...       ...
9999995  0.204886  0.351324  0.353573  0.409395
9999996  0.204886  0.351324  0.353573  0.409395
9999997  0.204886  0.351324  0.353573  0.409395
9999998  0.204886  0.351324  0.353573  0.409395
9999999  0.204886  0.351324  0.353573  0.409395

[10000000 rows x 4 columns]

In [9]:
nstep = 10000000
param_length = 4

In [10]:
#m =ps.iloc[:,0].tolist()
params = np.zeros((nstep, param_length))
for i in range(param_length):
    params[:, i] = list(df1.iloc[:, i])

print(np.shape(params))

(10000000, 4)


In [11]:
print(check_normalize(params))

True


In [12]:
params = params[500000:-1, :]
np.shape(params)

(9499999, 4)

In [13]:
low = [1E3, 1E37, 1E3, 0.05]
up = [1E6, 5E40, 5E6, 2]

In [14]:
params_denormalized = denormalize(np.asarray(params.T)[np.newaxis]) #normalizing the values -  converting to 2D array
params_denormalized = np.reshape(params_denormalized, (params_denormalized.shape[2], params_denormalized.shape[1]))

In [15]:
np.shape(params_denormalized)

(9499999, 4)

In [16]:
check_limits(params_denormalized, low, up)

(False, 23749996)

In [17]:
params_denormalized

array([[4.07247737e+03, 4.07247737e+03, 4.07247737e+03, 4.07247737e+03],
       [4.07247737e+03, 4.07247737e+03, 4.07247737e+03, 4.07247737e+03],
       [4.07247737e+03, 4.07247737e+03, 4.07247737e+03, 4.07247737e+03],
       ...,
       [8.48319806e-01, 8.48319806e-01, 8.48319806e-01, 8.48319806e-01],
       [8.48319806e-01, 8.48319806e-01, 8.48319806e-01, 8.48319806e-01],
       [8.48319806e-01, 8.48319806e-01, 8.48319806e-01, 8.48319806e-01]])

# Draw Samples

In [18]:
chain = params_denormalized
for i in range(chain.shape[1]): # ommiting the mean from the chain
    chain[:,i] = chain[:,i] - np.mean(chain[:,i])

mycov=(chain.T@chain)/chain.shape[0]

In [19]:
def draw_samples(cov,n):
    m = cov.shape[0]
    mat = np.random.randn(m,n)
    #print(mat)
    L = np.linalg.cholesky(cov)
    print(L)
    #the shape of the output is: number of samples * number of params
    return (L@mat).T

n = 100
#n= 100000
#there is a huge bug here! we can not have negative values in our samples but the np.random.randn creats negative values!
#That's why we encounter negative velues for params which is definately incorrect!
#By mathematical definition, gaussian can give negative results!

samples = draw_samples(mycov,n)
cov2 = samples.T@samples/n

[[8.76532535e+37 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [8.76531415e+37 1.34118687e+35 0.00000000e+00 0.00000000e+00]
 [8.76530578e+37 1.30712905e+35 1.46157438e+35 0.00000000e+00]
 [8.76530113e+37 1.29394083e+35 1.41510480e+35 1.48513792e+35]]


In [20]:
#print(samples)
#There is another huge bug here! The returned samples does not respect the resonable range for each param!

In [21]:
samples_normalized = normalize(np.asarray(samples.T)[np.newaxis]) #normalizing the guess values -  converting to 2D array
samples_normalized = np.reshape(samples_normalized, (samples_normalized.shape[2], samples_normalized.shape[1]))

<ipython-input-4-0ca7f9420f91>:10: RuntimeWarning: invalid value encountered in log10
  op[:,i] = (np.log10(arr[:,i]/ares_params[i][1]))/(np.log10(ares_params[i][2]/ares_params[i][1]))


In [22]:
np.shape(samples_normalized)

(100, 4)

In [23]:
check_limits(samples_normalized, low, up)

(False, 381)

In [24]:
np.savetxt('samples.txt', samples_normalized)

In [25]:
print(mycov)
print(cov2)

[[7.68309284e+75 7.68308303e+75 7.68307569e+75 7.68307162e+75]
 [7.68308303e+75 7.68309120e+75 7.68308341e+75 7.68307916e+75]
 [7.68307569e+75 7.68308341e+75 7.68309699e+75 7.68309207e+75]
 [7.68307162e+75 7.68307916e+75 7.68309207e+75 7.68310922e+75]]
[[6.10877297e+75 6.10957984e+75 6.11158847e+75 6.11092507e+75]
 [6.10957984e+75 6.11040366e+75 6.11241285e+75 6.11175135e+75]
 [6.11158847e+75 6.11241285e+75 6.11444382e+75 6.11378116e+75]
 [6.11092507e+75 6.11175135e+75 6.11378116e+75 6.11314381e+75]]


In [26]:
diff = mycov - cov2
diff_r = (diff/mycov)
print(diff_r)

[[0.20490705 0.20480101 0.20453882 0.20462474]
 [0.20480101 0.20469463 0.20443232 0.20451798]
 [0.20453882 0.20443232 0.20416938 0.20425512]
 [0.20462474 0.20451798 0.20425512 0.20433985]]
